# MODEL-PROJECT: The Black-Scholes(-Merton)-Model

**Before we start please install these packages in your VScode terminal (if not already installed)**

To do that just remove the '#' in the codecell below and run the code once. Be sure to put the '#' back in before you continue as to not run into problems when rerunning the code.

In [74]:
# !pip install yfinance
# !pip install ipywidgets
# !pip install dash
# !pip install dash-core-components

Imports and set magics:

In [75]:
import datetime
import numpy as np
import sympy as sp
import yfinance as yf
import ipywidgets as widgets
import matplotlib.pyplot as plt
from IPython.display import display
from scipy.stats import norm
from scipy import optimize
from dash import dcc

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Model description

The Black-Scholes model is a mathematical model for pricing European-style options. It's widely used in the financial industry, and it has significantly impacted the field of financial economics. The model assumes that the price of the underlying asset follows a geometric Brownian motion, and it derives a closed-form solution for the price of said asset. It provides a theoretical estimate of the value of a call or put option based on the following parameters:

-The current price of the underlying asset (S)  
-The strike price of the option (K)  
-The time to maturity (T)  
-The risk-free interest rate (r)  
-The volatility of the underlying asset (σ)  

Deriving the Black-Scholes PDE for a call-option on a non-dividend paying stock with strike K and maturity T. We assume that the stock price follows a geometric Brownian motion so that:

$ d (S_t) = μS_t dt + σS_tdW_t $ (i)

where $ W_t $ is a standard Brownian motion. We also assume that interest rates are constant so that 1 unit of currency invested in the cash account at time 0 will be worth $ B_t := exp(rt) $ at time t. We will denote by $ C(S, t) $ the value of the call option at time t. By Itˆo’s lemma we know that:

$ dC(S, t) = μS_t\frac{∂ C}{∂ S} + \frac{∂ C}{∂ t} + \frac{1}{2}σ^2S^2\frac{∂^2C}{∂S^2} dt + σS_t\frac{∂ C}{∂ S}dW_t$ (ii)

Let us now consider a self-financing trading strategy where at each time t we hold $x_t$ units of the cash account and $y_t$ units of the stock. Then $P_t$, the time t value of this strategy satisfies:

$P_t = x_tB_t + y_tS_t $ (iii)

We will choose $x_t$ and $y_t$ in such a way that the strategy replicates the value of the option. The self-financing assumption implies that:

$ dP_t = x_tdB_t + y_tdS_t$  (iv)  
$      = rx_tB_tdt + y_t(μS_tdt + σS_tdW_t) $  
$      = (rx_tB_t + y_tμS_t)dt + y_tσS_tdW_t $ (v)

We can equate terms in (ii)) with the corresponding terms in (v) to obtain:

$ y_t = \frac{∂ C}{∂ S} $  (vi)

$ rx_tB_t = \frac{∂ C}{∂ t} + \frac{1}{2}σ^2S^2\frac{∂^2C}{∂S^2} $  (vii)

If we set $C_0 = P_0$, the initial value of our self-financing strategy, then it must be the case that $C_t = P_t$ for all t since C and P have the same dynamics. This is true by construction after we equated terms in (ii) with the corresponding terms in (v). Substituting (vi) and (vii) into (iii) we obtain:


$ rS_t\frac{∂ C}{∂ S} + \frac{∂ C}{∂ t} + \frac{1}{2}σ^2S^2\frac{∂^2C}{∂S^2} - rC = 0 $ (viii) 


the Black-Scholes PDE. In order to solve (viii) boundary conditions must also be provided. In the case of the call option, these conditions are as follows:

$ C(S, T) = max(S-K, 0), C(0, t) = 0 $ for all t and $C(S, t) ⭢ S $ $as$ $ S ⭢ ∞ $  

We are finally able to arrive at the final formula for a european style call option as the solution to equation (viii):

$ C(S, K, T, r, σ) = S N(d1) - K e^{-rT} N(d2) $ 

where:

$ d1 = \frac{log(\frac{S}{K}) + (r + \frac{σ^2}{2})T}{σ\sqrt{T}} $

$ d2 = d1 - σ\sqrt{T} $  

and $N(x)$ is the cumulative distribution function of the standard normal distribution.

Equivalenty, from what we call the put-call parity we can also easily compute the price of a European put option using the formula for a call. By doing so we arrive at the following:

$ P(S, K, T, r, σ) = K e^{-rT} N(-d2) - S N(-d1) $  
  
  
In this project, we'll implement the Black-Scholes model using Python and perform various analyses, including numerical solutions for different real world assets and visualizations.


## Analytical solution

The Black-Scholes model .....

In [76]:
# Define the Black-Scholes Model symbols
S, K, T, r, sigma = sp.symbols('S K T r sigma')

# Define the Black-Scholes Model equation
d1 = ((r + 0.5*sigma**2)*T + sp.log(S/K)) / (sigma*sp.sqrt(T))
d1

(T*(r + 0.5*sigma**2) + log(S/K))/(sqrt(T)*sigma)

In [77]:
d2 = d1 - sigma*sp.sqrt(T)
d2

-sqrt(T)*sigma + (T*(r + 0.5*sigma**2) + log(S/K))/(sqrt(T)*sigma)

In [78]:
# Define lambdified functions for d1 and d2
d1_lambdified = sp.lambdify((S, K, T, r, sigma), d1, 'numpy')
d2_lambdified = sp.lambdify((S, K, T, r, sigma), d2, 'numpy')

# Define the Black-Scholes Model equation
def black_scholes(S, K, T, r, sigma, option_type='call'):
    d1 = d1_lambdified(S, K, T, r, sigma)
    d2 = d2_lambdified(S, K, T, r, sigma)
    
    if option_type == 'call':
        price = S*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)
    else:
        price = K*np.exp(-r*T)*norm.cdf(-d2) - S*norm.cdf(-d1)
    
    return price

## Numerical solution

We can also solve the Black-Scholes model numerically using optimization algorithms. In this case, we'll use the Newton-Raphson method.

In [79]:
# Define the initial values for K, S, T, r, and sigma
K = 100
S = 100
T = 1
r = 0.05
sigma = 0.2

# Define a range of strike prices
stock_prices = np.linspace(1, 200, 100)

# Create a function to update the plot with the new value of sigma
def update_plot(K, T, r, sigma):
    call_prices = [black_scholes(S, K, T, r, sigma, option_type='call') for S in stock_prices]
    put_prices = [black_scholes(S, K, T, r, sigma, option_type='put') for S in stock_prices]
    
    plt.figure(figsize=(10, 6))
    plt.plot(stock_prices, call_prices, label="Call prices")
    plt.plot(stock_prices, put_prices, label="Put prices")
    plt.title(f"Black-Scholes Model for K={K}, T={T}, r={r}, sigma={sigma}")
    plt.xlabel("Stock Price")
    plt.ylabel("Option Price")
    plt.legend()
    plt.show()

# Create a slider widget for S
K_slider = widgets.FloatSlider(
    value=K,
    min=S * 0.5,
    max=S * 1.5,
    step=1,
    description='Strike price "K":',
    continuous_update=False
)    

# Create a slider widget for T
T_slider = widgets.FloatSlider(
    value=T,
    min=1,
    max=10,
    step=1,
    description='Time to maturity "T":',
    continuous_update=False
)

# Create a slider widget for r
r_slider = widgets.FloatSlider(
    value=r,
    min=0,
    max=0.2,
    step=0.005,
    description='Risk free rate "r":',
    continuous_update=False
)

# Create a slider widget for sigma
sigma_slider = widgets.FloatSlider(
    value=sigma,
    min=0.01,
    max=1,
    step=0.01,
    description='Sigma:',
    continuous_update=False
)

# Create an output widget to display the plot
out = widgets.Output()

# Display the interactive plot
widgets.interact(update_plot, K=K_slider, T=T_slider, r=r_slider, sigma=sigma_slider)
display(out)


interactive(children=(FloatSlider(value=100.0, continuous_update=False, description='Strike price "K":', max=1…

Output()

# Further analysis

Now, let's analyze how the model changes with different parameter values and visualize the results.

In [80]:
# Creating a dictionary of interesting stocks and their respective ticker symbols
company_list = ["Apple", "Microsoft", "Google", "Amazon", "Tesla", "Netflix", "Berkshire Hathaway", 
                "JPMorgan Chase", "Johnson & Johnson", "Visa", "Walmart", "UnitedHealth Group", 
                "Procter & Gamble", "NVIDIA", "Home Depot"] 
ticker_list = ["AAPL", "MSFT", "GOOG", "AMZN", "TSLA", "NFLX", "BRK-B", "JPM", "JNJ", "V", "WMT", 
               "UNH", "PG", "NVDA", "HD"]
company_ticker_dict = {company: ticker for company, ticker in zip(company_list, ticker_list)}


# Get today's date and the date exactly one year ago in string format 
today = datetime.date.today()
yesterday = today - datetime.timedelta(days=1)
yesterday_last_year = today - datetime.timedelta(days=366)
yesterday_str = yesterday.strftime("%Y-%m-%d")
yesterday_one_year_ago_str = yesterday_last_year.strftime('%Y-%m-%d')


# Loop through the list of stock-tickers, calculate the historical volatility and store in a dictionary
historical_sigma_dict = {}                                                                      
for ticker in ticker_list:
    data = yf.download(ticker, start=yesterday_one_year_ago_str, end=yesterday_str) # Download price data 
    trading_days = len(data)                                                        # Determine number of trading days
    returns = data["Close"].pct_change()                                            # Calculate the daily returns
    std_dev = np.std(returns)                                                       # Calculate standard deviation of daily returns
    annual_volatility = std_dev * np.sqrt(trading_days)                             # Calculate annualized volatility (sigma)
    historical_sigma_dict[ticker] = annual_volatility                               # Add results to dictionary









# Creating a Function to plot the Black-Scholes model for a given stock ticker
def plot_black_scholes(company, T_val=T, r_val=r, sigma_val=sigma):
    with out:
        try:
            # Get stock data
            stock_ticker = company_ticker_dict[company]
            stock_data = yf.download(stock_ticker, progress=False)
            stock_price = stock_data.iloc[-1]["Close"]
            #stock_sigma = 

            # Calculate call and put prices for a range of strike prices
            strike_prices = np.linspace(stock_price * 0.5, stock_price * 1.5, 100)
            call_prices = [black_scholes(stock_price, K, T_val, r_val, sigma_val, option_type='call') for K in strike_prices]
            put_prices = [black_scholes(stock_price, K, T_val, r_val, sigma_val, option_type='put') for K in strike_prices]

            # Clear the output and create the plot
            out.clear_output(wait=True)
            plt.figure(figsize=(10, 6))
            plt.plot(strike_prices, call_prices, label="Call prices")
            plt.plot(strike_prices, put_prices, label="Put prices")

            plt.title(f"Black-Scholes Model for {company}")
            plt.xlabel("Strike Price")
            plt.ylabel("Option Price")
            plt.legend()
            plt.show()
        except IndexError as e:
            print(f"Error: {e}. Data for {company} might not be available or the stock might be delisted.")


# Create a dropdown menu for company selection and sliders for the widgets 
company_dropdown = widgets.Dropdown(
    options=company_list,
    value=company_list[0],
    description="Company:",
)

T_slider2 = widgets.FloatSlider(
    value=T,
    min=1,
    max=10,
    step=1,
    description='Time to maturity "T":',
    continuous_update=False
)

r_slider2 = widgets.FloatSlider(
    value=r,
    min=0,
    max=0.2,
    step=0.005,
    description='Risk free rate "r":',
    continuous_update=False
)

sigma_slider2 = widgets.FloatSlider(
    value=sigma,
    min=0.01,
    max=1,
    step=0.01,
    description='Sigma:',
    continuous_update=False
)


# Create an Output widget and display the interactive plot
out = widgets.Output()
widgets.interact(plot_black_scholes, company=company_dropdown, T_val=T_slider2, r_val=r_slider2, sigma_val=sigma_slider2)
display(out)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

interactive(children=(Dropdown(description='Company:', options=('Apple', 'Microsoft', 'Google', 'Amazon', 'Tes…

Output()

# Extension of the Model - Dividends

In the standard Black & Scholes model we have covered above, dividends are assumed to be zero and are not incorporated into the pricing of options. However, we can extend on this basic model by accomodating for dividends and seeing how this change affects the prices of both call and put options.

In the presence of dividends, the drift rate is adjusted to reflect the present value of future dividends. Specifically, the drift rate becomes:

$ (r - q) $

where $r$ is the risk-free interest rate and $q$ is the continuous dividend yield.

When dividends are included, the Black-Scholes model assumes that the stock price follows a geometric Brownian motion with the drift rate adjusted by the dividend yield. 

With this extension of the model, we can show how the dynamics of the option price satisfy:

$ C(S, K, T, r, σ) = S  e^{-qT} N(d1) - K e^{-rT} N(d2) $   

$ P(S, K, T, r, σ) = K e^{-rT} N(-d2) - S e^{-qT} N(-d1) $  

where:

$ d1 = \frac{log(\frac{S}{K}) + (r-q + \frac{σ^2}{2})T}{σ\sqrt{T}} $  

$ d2 = d1 - σ\sqrt{T} $  

Notice that the dividend yield, q, appears in the formula for $d1$ and in the term $e^{-qT}$, which adjusts the stock price for the present value of the dividends. This term reflects the idea that a dividend payment reduces the value of the stock, as it represents cash flow leaving the company.

Essentially,

Incorporating dividends into the Black-Scholes model allows for a more accurate calculation of option prices, especially for options on stocks that pay significant dividends. This is important for investors who want to make informed decisions about buying or selling options based on their expected value and risk.

In [81]:
# Define the Black-Scholes Model equation
def black_scholes_div(S, K, T, r, q, sigma, option_type='call'):
    d1 = ((r-q + 0.5*sigma**2)*T + np.log(S/K)) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    
    if option_type == 'call':
        price = S*np.exp(-q*T)*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)
    else:
        price = K*np.exp(-r*T)*norm.cdf(-d2) - S*np.exp(-q*T)*norm.cdf(-d1)
    
    return price

#For simplicity, define the average dividend yield for the S&P 500 index, 
#which represents a broad range of large-cap U.S. companies, as around 1.5%.
q = 0.015

# Create a function to update the plot with the new value of sigma
def update_plot_div(K, T, r, q, sigma):
    call_prices = [black_scholes_div(S, K, T, r, q, sigma, option_type='call') for S in stock_prices]
    put_prices = [black_scholes_div(S, K, T, r, q, sigma, option_type='put') for S in stock_prices]
    
    plt.figure(figsize=(10, 6))
    plt.plot(stock_prices, call_prices, label="Call prices")
    plt.plot(stock_prices, put_prices, label="Put prices")
    plt.title(f"Black-Scholes Model for K={K}, T={T}, r={r}, q={q}, sigma={sigma}")
    plt.xlabel("Stock Price")
    plt.ylabel("Option Price")
    plt.legend()
    plt.show()

# Create a slider widget for S
K_slider2 = widgets.FloatSlider(
    value=K,
    min=S*0.5,
    max=S*1.5,
    step=1,
    description='Strike price "K":',
    continuous_update=False
)    

# Create a slider widget for T
T_slider3 = widgets.FloatSlider(
    value=T,
    min=1,
    max=10,
    step=1,
    description='Time to maturity "T":',
    continuous_update=False
)

# Create a slider widget for r
r_slider3 = widgets.FloatSlider(
    value=r,
    min=0,
    max=0.2,
    step=0.005,
    description='Risk free rate "r":',
    continuous_update=False
)

# Create a slider widget for sigma
sigma_slider3 = widgets.FloatSlider(
    value=sigma,
    min=0.01,
    max=1,
    step=0.01,
    description='Sigma:',
    continuous_update=False
)

# Create a slider widget for q
q_slider = widgets.FloatSlider(
    value=q,
    min=0,
    max=0.15,
    step=0.01,
    description='Dividend Yield:',
    continuous_update=False
)

# Create an output widget to display the plot
out = widgets.Output()

# Display the interactive plot
widgets.interact(update_plot_div, K=K_slider2, T=T_slider3, r=r_slider3, sigma=sigma_slider3,  q=q_slider)
display(out)


interactive(children=(FloatSlider(value=100.0, continuous_update=False, description='Strike price "K":', max=1…

Output()

When a stock pays a dividend, the price of the stock is expected to drop by an amount equal to the dividend paid. This drop in the stock price reduces the potential profit that can be earned from holding the call option. 

As a result, when the dividend yield of the underlying stock increases, the value of the call option decreases. Conversely, when the dividend yield decreases, the value of the call option increases. 

The drop in the stock price increases the potential profit that can be earned from holding the put option. 

As a result, when the dividend yield of the underlying stock increases, the value of the put option increases. Conversely, when the dividend yield decreases, the value of the put option decreases.

One key result, which can be inferred from simply looking at the formula and the interaction of the variables, is how a longer time to maturity + an increasing dividend yield has a drastic effect on stock price and by extension, the call/put price as well (we can assume that the timeframe is at the very least on a per quarter basis as that is a good representation of dividend payoffs).


# Conclusion

Additionally, in this version of the project, we have created an interactive plot to visualize the Black-Scholes option prices for 15 different European stocks. The stocks were chosen to represent a diverse range of industries and countries. The interactive plot allows users to select a stock from a dropdown menu and view the corresponding option prices for the call and put options.